In [1]:
import json
import numpy as np


In [2]:
examplesJson="example.json"
batchSize=2
map_size=32

In [3]:


X = np.array(json.load(open(examplesJson)))
# print(X)
print(X.shape)
z_dims = 10  # Numer different title types

num_batches = X.shape[0] / batchSize

print("SHAPE ", X.shape)
X_onehot = np.eye(z_dims, dtype='uint8')[X]

X_onehot = np.rollaxis(X_onehot, 3, 1)
print("SHAPE ", X_onehot.shape)  # (173, 14, 28, 16)

X_train = np.zeros((X.shape[0], z_dims, map_size, map_size))*2

X_train[:, 2, :, :] = 1.0  # Fill with empty space

# Pad part of level so its a square
X_train[:X.shape[0], :, :X.shape[1], :X.shape[2]] = X_onehot

(173, 14, 28)
SHAPE  (173, 14, 28)
SHAPE  (173, 10, 14, 28)


In [4]:
levelPath = "../marioaiDagstuhl/data/mario/levels/mario-1-1.txt"
windowSize = (14, 28)

mapping = {"X": 0, "S": 1, "-": 2, "?": 3, "Q": 4,
           "E": 5, "<": 6, ">": 7, "[": 8, "]": 9}

reverse_mapping = {v: k for k, v in mapping.items()}

In [5]:
# Define the sliding window function
def sliding_window(arr, window_size):
    windows = []
    for i in range(arr.shape[0] - window_size[0] + 1):
        for j in range(arr.shape[1] - window_size[1] + 1):
            window = arr[i:i + window_size[0], j:j + window_size[1]]
            windows.append(window)
    return np.array(windows)

with open(levelPath, 'r') as f:
    level = f.readlines()
    level = [x.strip() for x in level]

mapped_level = [[mapping[char] for char in line] for line in level]

mapped_level = np.array(mapped_level)

print(mapped_level.shape)

# Apply the sliding window
windows = sliding_window(mapped_level[:,:-2], windowSize)

# Print the shape of the resulting arrays list
print(f"Number of resulting arrays: {len(windows)}")
print(f"Shape of each array: {windows[0].shape}")

(14, 202)
Number of resulting arrays: 173
Shape of each array: (14, 28)


In [6]:
reversed_level_new = [[reverse_mapping[num] for num in line] for line in windows[172]]
reversed_level_orig = [[reverse_mapping[num] for num in line] for line in X[172]]


# Print the reversed level to verify
for line in reversed_level_new:
    print(''.join(line))

for line in reversed_level_orig:
    print(''.join(line))

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------XX----------
---------------XXX----------
--------------XXXX----------
-------------XXXXX----------
------------XXXXXX----------
-----------XXXXXXX----------
-------<>-XXXXXXXX----------
--EE---[]XXXXXXXXX--------X-
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------XX----------
---------------XXX----------
--------------XXXX----------
-------------XXXXX----------
------------XXXXXX----------
-----------XXXXXXX----------
-------<>-XXXXXXXX----------
--EE---[]XXXXXXXXX--------X-
XXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [7]:
are_equal = np.array_equal(windows, X)
print(f"Are the arrays exactly equal? {are_equal}")

# If the arrays are not equal, find and print the differences
if not are_equal:
    # Find the indices where the arrays differ
    differences = np.where(windows != X)
    
    # Print the indices and the corresponding values from both arrays
    for index in zip(*differences):
        print(f"Index: {index}, windows value: {windows[index]}, X value: {X[index]}")

Are the arrays exactly equal? True


## Load to JSON

In [8]:
windows[0].shape

(14, 28)

In [20]:
save_path = "windows.json"
windows_json = json.dumps(windows.tolist())

with open(save_path, 'w') as f:
    f.write(windows_json)

In [21]:
# Load JSON data from the first file
with open('windows.json', 'r') as f:
    data1 = json.load(f)

# Load JSON data from the second file
with open('example.json', 'r') as f:
    data2 = json.load(f)

# Compare the JSON data
if data1 == data2:
    print("The JSON files are identical.")
else:
    print("The JSON files are different.")
    
    # Optionally, print the differences
    # from deepdiff import DeepDiff
    # differences = DeepDiff(data1, data2, ignore_order=True)
    # print("Differences:", differences)

The JSON files are identical.
